In [15]:
import numpy as np 
import pandas as pd 
import seaborn as snb
import matplotlib.pyplot as plt
%matplotlib inline

import os 
os.environ['CUDA_VISIBLE_DEVICES'] = '-1' # forces tf to run on cpu 

In [16]:
train_df = pd.read_csv('data/train.csv')

train_df = train_df.drop('Segmentation' , axis=1)
train_df.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6
4,462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6


In [17]:
train_df.isnull().sum()

ID                   0
Gender               0
Ever_Married       140
Age                  0
Graduated           78
Profession         124
Work_Experience    829
Spending_Score       0
Family_Size        335
Var_1               76
dtype: int64

In [18]:
# replaces NaN values in numerical columns with the mode of each respective column.
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
col_num = train_df.select_dtypes(include=numerics)

for col in col_num:
    train_df[col].replace([np.nan], train_df[col].mode()[0], inplace=True)